In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ase.io import read
import matplotlib.pyplot as plt
from ase.visualize.plot import plot_atoms
import mogli

In [2]:
bandgaps = np.loadtxt('Datasets/bandgaps.csv')
cods = np.loadtxt('Datasets/CODids.csv', dtype=int)
materials = mogli.read('Datasets/structures.xyz')

In [1]:
import os
path = 'Datasets/3d'
if not os.path.exists(path):
    os.makedirs(path)
distance = 25
for i, molecule in enumerate(materials):
    # frente 
    mogli.export(molecule,f"{path}/enfrente/enfrente_{i}.png",bonds_param=1.75,camera=((distance, 0, 0),(0, 0, 0),(0 , 1 , 0)), width=500, height=500)
    # atras
    mogli.export(molecule,f"{path}/atras/atras_{i}.png",bonds_param=1.75,camera=((-distance, 0, 0),(0, 0, 0),(0, 1 , 0)), width=500, height=500)

    # 90 grados derecha 
    mogli.export(molecule,f"{path}/Izquierda/Izquierda_{i}.png",bonds_param=1.75,camera=((0, 0, distance),(0, 0, 0),(0,1, 0)), width=500, height=500)
    # 90 grados izquierda
    mogli.export(molecule,f"{path}/Derecha/Derecha_{i}.png",bonds_param=1.75,camera=((0, 0, distance),(0, 0, 0),(0, -1, 0)), width=500, height=500)

    # abajo
    mogli.export(molecule,f"{path}/abajo/abajo_{i}.png",bonds_param=1.75,camera=((0, -distance, 0),(0, 0, 0),(1, 0, 0)), width=500, height=500)
    # arriba
    mogli.export(molecule,f"{path}/arriba/arriba_{i}.png",bonds_param=1.75,camera=((0, distance, 0),(0, 0, 0),(-1, 0, 0)), width=500, height=500)
    

NameError: name 'materials' is not defined